## [Sudoku](https://leetcode.com/problems/sudoku-solver/)
- Solve a 9x9 sudoku grid
    + Row: Each of the digits `1-9` must occur exactly once in each row.
    + Col: Each of the digits `1-9` must occur exactly once in each column.
    + Subbox: Each of the the digits `1-9` must occur exactly once in each of the 9 3x3 sub-boxes of the grid.
- Example

| Board                      |  Solution                 |
|:--------------------------:|:-------------------------:|
|![](./img/7.png)            |  ![](./img/8.png)         |

```C++
vector<vector<char>> board = {
    {'5','3','.','.','7','.','.','.','.'},
    {'6','.','.','1','9','5','.','.','.'},
    {'.','9','8','.','.','.','.','6','.'},
    {'8','.','.','.','6','.','.','.','3'},
    {'4','.','.','8','.','3','.','.','1'},
    {'7','.','.','.','2','.','.','.','6'},
    {'.','6','.','.','.','.','2','8','.'},
    {'.','.','.','4','1','9','.','.','5'},
    {'.','.','.','.','8','.','.','7','9'}};
```

#### Solution 1 - Backtracking - Check satisfaction O(n)

```C++
// solve();
int n=9, m=9;
bool satisfy(int r, int c, char ch) {
    // Row
    for(int j=0; j<m; ++j) {
        if(board[r][j] == ch)
            return false;
    }

    // Col
    for(int i=0; i<n; ++i) {
        if(board[i][c] == ch)
            return false;
    }

    // Subbox
    for(int i=r/3*3; i<(r+3)/3*3; ++i) for(int j=c/3*3; j<(c+3)/3*3; ++j) {
        if(board[i][j] == ch)
            return false;
    }
    return true;
}
bool solve() {
    for(int r=0; r<n; ++r) for(int c=0; c<m; ++c) {
        // Skip fixed numbers
        if(board[r][c] != '.') continue;
        
        // Find a feasible num for each unfill cell
        for(char ch='1'; ch<='9'; ++ch) {
            if(satisfy(r,c,ch)) {
                board[r][c] = ch;
                if(solve()) return true;
                board[r][c] = '.';
            }
        }
        
        // Fail in all attemps
        return false;
    }
    
    // All feasible
    return true;
}
```

#### Solution 2 -  Backtracking - Check satisfaction O(n)
```C++
// solve(0)
int n=9, m=9;
bool satisfy(int r, int c, char ch) {
    // Row
    for(int i=0; i<n; ++i) {
        if(board[i][c] == ch)
            return false;
    }

    // Col
    for(int j=0; j<m; ++j) {
        if(board[r][j] == ch)
            return false;
    }

    // Subbox
    for(int i=r/3*3; i<(r+3)/3*3; ++i) for(int j=c/3*3; j<(c+3)/3*3; ++j) {
        if(board[i][j] == ch)
            return false;
    }
    return true;
}
bool solve(int cell) {
    // All feasible
    if(cell == n*m) {
        return true;
    }

    // map cell -> (r,c)
    int r = cell / n;
    int c = cell % n;

    // Skip fixed numbers
    if(board[r][c] != '.') 
        return solve(cell+1);

    // Find a feasible num for each unfill cell 
    for(char ch='1'; ch<='9'; ++ch) {
        if(satisfy(r,c,ch)) {
            board[r][c] = ch;
            if(solve(cell+1)) return true;
            board[r][c] = '.';
        }
    }

    // Fail in all attemps
    return false;
}
```

#### Solution 3 - Constraint Programming - Check satisfaction O(1)

```C++
int n=9, m=9;
vector<int> row_constraints;
vector<int> col_constraints;
vector<vector<int>> subbox_constraints;
bool satisfy(int r, int c, char ch) {
    // Row
    if(row_constraints[r] & (1 << (ch - '0')))
        return false;

    // Col
    if(col_constraints[c] & (1 << (ch - '0')))
        return false;

    // Subbox
    if(subbox_constraints[r/3][c/3] & (1 << (ch - '0')))
        return false;

    return true;
}
bool solve(int cell) {
    if(cell == n*m) {
        return true;
    }

    // map cell -> (r,c)
    int r = cell / n;
    int c = cell % n;

    // Skip fixed numbers
    if(board[r][c] != '.') 
        return solve(cell+1);

    // Find a feasible num for each unfill cell 
    for(char ch='1'; ch<='9'; ++ch) {
        if(satisfy(r,c,ch)) {
            board[r][c] = ch;
            row_constraints[r] |= (1 << (ch - '0'));
            col_constraints[c] |= (1 << (ch - '0'));
            subbox_constraints[r/3][c/3] |= (1 << (ch - '0'));

            if(solve(cell+1)) return true;

            board[r][c] = '.';
            row_constraints[r] &= ~(1 << (ch - '0'));
            col_constraints[c] &= ~(1 << (ch - '0'));
            subbox_constraints[r/3][c/3] &= ~(1 << (ch - '0'));
        }
    }

    // Fail in all attemps
    return false;
}
void solveSudoku() {
    row_constraints.assign(n, 0);
    col_constraints.assign(m, 0);
    subbox_constraints.assign(n/3, vector<int>(m/3, 0));

    // Setup constraints
    for(int r=0; r<n; ++r) for(int c=0; c<m; ++c) {
        if(board[r][c] == '.') 
            continue;
        row_constraints[r] |= (1 << (board[r][c] - '0'));
        col_constraints[c] |= (1 << (board[r][c] - '0'));
        subbox_constraints[r/3][c/3] |= (1 << (board[r][c] - '0'));
    }

    // Solve
    solve(0);
}
```